In [ ]:
import torch
print(torch.__version__)

1.13.1+cu116


# Tutorial

In [ ]:
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose

In [ ]:
## Download Dataset

training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [ ]:
## Dataloader. tf.data.dataset과 유사.

batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W] : ", X.shape)
    print("Shape of y : ", y.shape, y.dtype)

    break

Shape of X [N, C, H, W] :  torch.Size([64, 1, 28, 28])
Shape of y :  torch.Size([64]) torch.int64


In [ ]:
## 학습에 사용할 장치를 설정.

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
## model

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(nn.Linear(28 * 28, 128),
                                               nn.ReLU(),
                                               nn.Dropout(0.2),
                                               nn.Linear(128, 10))
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)

        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [ ]:
## Loss & Optimizer

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
## Training function

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) ## train_dataloader에 들어 있는 dataset의 양(len).
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device) ## X.shape : torch.Size([64, 1, 28, 28]) torch.Size([64])

        ## loss 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        ## 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0: ## 100번째 step마다 상태 출력.
            loss, current = loss.item(), batch * len(X) ## 100 * 64
            print(f"loss : {loss:>7f} [{current:>5d}/{size:>5d}]")

In [ ]:
## Test function

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad(): ## 역전파 하지 않을 것이다.
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error : \n Accuracy : {(100 * correct):>0.1f}%, Avg loss : {test_loss:>8f} \n")

In [ ]:
epochs = 10

for epoch in range(epochs):
    print(f"Epoch {epoch + 1} \n ---------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print("Finished")

Epoch 1 
 ---------------------------------
0 64
loss : 2.321279 [    0/60000]
100 64
loss : 0.488264 [ 6400/60000]
200 64
loss : 0.307639 [12800/60000]
300 64
loss : 0.379505 [19200/60000]
400 64
loss : 0.275043 [25600/60000]
500 64
loss : 0.337635 [32000/60000]
600 64
loss : 0.186348 [38400/60000]
700 64
loss : 0.376723 [44800/60000]
800 64
loss : 0.266436 [51200/60000]
900 64
loss : 0.332404 [57600/60000]
Test Error : 
 Accuracy : 94.2%, Avg loss : 0.192583 

Epoch 2 
 ---------------------------------
0 64
loss : 0.167827 [    0/60000]
100 64
loss : 0.184269 [ 6400/60000]
200 64
loss : 0.115567 [12800/60000]
300 64
loss : 0.175174 [19200/60000]
400 64
loss : 0.155731 [25600/60000]
500 64
loss : 0.238123 [32000/60000]
600 64
loss : 0.065203 [38400/60000]
700 64
loss : 0.200087 [44800/60000]
800 64
loss : 0.189300 [51200/60000]
900 64
loss : 0.184771 [57600/60000]
Test Error : 
 Accuracy : 96.1%, Avg loss : 0.129505 

Epoch 3 
 ---------------------------------
0 64
loss : 0.097790 [

# Tensor

In [ ]:
## Tensor
data = [[1, 2], [3, 4]]
x_data = torch.tensor(data)
print(x_data)

np_array = np.array(data)
x_np = torch.tensor(np_array)
print(x_np)

tensor([[1, 2],
        [3, 4]])
tensor([[1, 2],
        [3, 4]])


In [ ]:
tensor_to_np = x_np.numpy()
print(tensor_to_np, type(tensor_to_np))

[[5 2]
 [3 4]] <class 'numpy.ndarray'>


In [ ]:
x_np[0, 0] = 5
print(x_np)
print(np_array) ## toch.tensor()는 tensor를 copy해서 값을 변경한다.

tensor([[5, 2],
        [3, 4]])
[[1 2]
 [3 4]]


In [ ]:
## as_tensor, from_numpy는 view를 만들기 때문에 원본 값까지도 바뀐다.
x_np_2 = torch.as_tensor(np_array)
x_np_3 = torch.from_numpy(np_array)

x_np_2[0, 1] = 10
x_np_3[1, 0] = 20
print(x_np_2)
print(x_np_3)
print(np_array)

tensor([[ 1, 10],
        [20,  4]])
tensor([[ 1, 10],
        [20,  4]])
[[ 1 10]
 [20  4]]


In [ ]:
## tensor.device는 해당 텐서가 어느 device에 올라가는지 확인할 수 있다.
tensor = torch.rand(3, 4)

print(tensor.shape)
print(tensor.dtype)
print(tensor.device)

torch.Size([3, 4])
torch.float32
cpu


In [ ]:
tensor = tensor.reshape(4, 3)
tensor = tensor.int() ## dtype 변환.
if torch.cuda.is_available():
    tensor = tensor.to("cuda")

print(tensor.shape)
print(tensor.dtype)
print(tensor.device)

torch.Size([4, 3])
torch.int32
cuda:0


In [ ]:
## Transpose
a = torch.arange(16).reshape(2, 2, 4)
print(a, a.shape)

tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7]],

        [[ 8,  9, 10, 11],
         [12, 13, 14, 15]]]) torch.Size([2, 2, 4])


In [ ]:
b = a.transpose(1, 2) ## np.swapaxes와 동일.
print(b, b.shape)

tensor([[[ 0,  4],
         [ 1,  5],
         [ 2,  6],
         [ 3,  7]],

        [[ 8, 12],
         [ 9, 13],
         [10, 14],
         [11, 15]]]) torch.Size([2, 4, 2])


In [ ]:
c = a.permute((2, 0, 1)) ## np.transpose와 동일.
print(c, c.shape)

tensor([[[ 0,  4],
         [ 8, 12]],

        [[ 1,  5],
         [ 9, 13]],

        [[ 2,  6],
         [10, 14]],

        [[ 3,  7],
         [11, 15]]]) torch.Size([4, 2, 2])


## Tensor operation

In [ ]:
x = torch.tensor([[1, 2], [3, 4]], dtype=torch.float32)
y = torch.tensor([[5, 6], [7, 8]], dtype=torch.float32)

print(x + y)
print(torch.add(x, y))

## new
print(x.add(y))
print(x)

## in-place : 연산한 값을 해당 텐서에 바로 저장한다.
print(x.add_(y))
print(x)

tensor([[ 6.,  8.],
        [10., 12.]])
tensor([[ 6.,  8.],
        [10., 12.]])
tensor([[ 6.,  8.],
        [10., 12.]])
tensor([[1., 2.],
        [3., 4.]])
tensor([[ 6.,  8.],
        [10., 12.]])
tensor([[ 6.,  8.],
        [10., 12.]])


In [ ]:
z = torch.arange(1, 11).reshape(2, 5)
print(z)

tensor([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10]])


In [ ]:
## Tensorflow는 tf.reduce_sum, pytorch는 numpy와 동일.
sum1 = torch.sum(z, axis=0)
sum2 = torch.sum(z, axis=1)
sum3 = torch.sum(z, axis=-1)

print(sum1, sum1.shape)
print(sum2, sum2.shape)
print(sum3, sum3.shape)

tensor([ 7,  9, 11, 13, 15]) torch.Size([5])
tensor([15, 40]) torch.Size([2])
tensor([15, 40]) torch.Size([2])


In [ ]:
## 텐서 복사
a = torch.arange(24).reshape(4, 6)
b = a.clone().detach()
print(a, a.shape)
print(b, b.shape)

tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23]]) torch.Size([4, 6])
tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23]]) torch.Size([4, 6])


In [ ]:
c = torch.cat([a, b], axis=0)
print(c, c.shape)

c = torch.cat([a, b], axis=-1)
print(c, c.shape)

tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23],
        [ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23]]) torch.Size([8, 6])
tensor([[ 0,  1,  2,  3,  4,  5,  0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11,  6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17, 12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23, 18, 19, 20, 21, 22, 23]]) torch.Size([4, 12])


In [ ]:
d = torch.stack([a, b], axis=0)
print(d, d.shape)

d = torch.stack([a, b], axis=-1)
print(d, d.shape)

tensor([[[ 0,  1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10, 11],
         [12, 13, 14, 15, 16, 17],
         [18, 19, 20, 21, 22, 23]],

        [[ 0,  1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10, 11],
         [12, 13, 14, 15, 16, 17],
         [18, 19, 20, 21, 22, 23]]]) torch.Size([2, 4, 6])
tensor([[[ 0,  0],
         [ 1,  1],
         [ 2,  2],
         [ 3,  3],
         [ 4,  4],
         [ 5,  5]],

        [[ 6,  6],
         [ 7,  7],
         [ 8,  8],
         [ 9,  9],
         [10, 10],
         [11, 11]],

        [[12, 12],
         [13, 13],
         [14, 14],
         [15, 15],
         [16, 16],
         [17, 17]],

        [[18, 18],
         [19, 19],
         [20, 20],
         [21, 21],
         [22, 22],
         [23, 23]]]) torch.Size([4, 6, 2])
